In [3]:
# Раскомментить и установить расширение

In [ ]:
# pip install ifcopenshell 


In [1]:
# pip install pandas

In [2]:
import ifcopenshell
import pandas as pd

No stream support: No module named 'lark'
/var/folders/rn/rl1dzql55sd8tt1fhxq5dlzm0000gn/T/ipykernel_25741/333723811.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
def extract_ifc_properties_to_dataframe(ifc_path, object_type, properties_list):
    # Загрузка IFC файла
    file = ifcopenshell.open(ifc_path)
    
    # Получение всех объектов заданного типа
    objects = file.by_type(object_type)
    
    # Подготовка данных для DataFrame
    data = []

    for obj in objects:
        obj_data = {'GlobalId': obj.GlobalId}
        for property_set in obj.IsDefinedBy:
            if property_set.is_a('IfcRelDefinesByProperties'):
                property_set_definition = property_set.RelatingPropertyDefinition
                if property_set_definition.is_a('IfcPropertySet'):
                    for (pset_name, prop_name) in properties_list:
                        if property_set_definition.Name == pset_name:
                            for property in property_set_definition.HasProperties:
                                if property.Name == prop_name and property.is_a('IfcPropertySingleValue'):
                                    # Добавляем значение свойства в данные объекта
                                    obj_data[f'{pset_name}.{prop_name}'] = property.NominalValue.wrappedValue
        data.append(obj_data)

    # Создание DataFrame
    df = pd.DataFrame(data)
    df.set_index('GlobalId', inplace=True)
    
    return df


In [9]:

# Пример использования функции
ifc_path = '12002-81-020601-КМ3.1.3.ifc'
object_type = 'IfcElement'
properties_list = [
    ('Tekla Quantity', 'Weight'),
    ('Pset_WallCommon', 'Height')
]

df = extract_ifc_properties_to_dataframe(ifc_path, object_type, properties_list)
df


,Tekla Quantity.Weight
GlobalId,
0cryUIbA55I9w6zXlNGfwV,420.60
0JG_ZbA9T1Q8XB5M4qoezx,38.02
2RTzw4jbf0FQmMYmColiZh,19.57
0D$inzAnnDXvy7fR5jDtv5,12.96
3_Vu0EUSLDoBPiwg2fsVcI,21.31
...,...
2JPJHsQLT1zOPsMqvJDdo9,70.76
3NKQ$V9k1F3gAKixq4dbqM,547.02
131ujOmhvA8QUlDsviILBl,63.91
